In [1]:
# Dependencies
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
import os
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import yfinance as yf

tickerStr = "AAPL"
ticker = yf.Ticker(tickerStr)
df = ticker.history(period='5y')
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-11-16,25.588095,26.245143,25.500796,26.231359,152426800,0.0,0.0
2015-11-17,26.401363,26.431230,26.033784,26.118788,110467600,0.0,0.0
2015-11-18,26.594344,26.991788,26.534612,26.945841,186698800,0.0,0.0
2015-11-19,27.026249,27.510994,26.824081,27.288149,173183200,0.0,0.0
2015-11-20,27.384641,27.550051,27.304233,27.407616,137148400,0.0,0.0
...,...,...,...,...,...,...,...
2020-11-09,120.500000,121.989998,116.050003,116.320000,154515300,0.0,0.0
2020-11-10,115.550003,117.589996,114.129997,115.970001,138023400,0.0,0.0
2020-11-11,117.190002,119.629997,116.440002,119.489998,112295000,0.0,0.0


In [3]:
df = df['Close'].values

In [4]:
df = df.reshape(-1, 1)
df.shape

(1259, 1)

In [5]:
dataset_train = np.array(df[:int(df.shape[0]*0.8)])
dataset_test = np.array(df[int(df.shape[0]*0.8)-50:])
#dataset_test = np.array(df[int(df.shape[0]*0.8):])
print(dataset_train.shape)
print(dataset_test.shape)

(1007, 1)
(302, 1)


In [7]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset_train = scaler.fit_transform(dataset_train)
dataset_train[:7]

array([[0.11896131],
       [0.11640431],
       [0.13519032],
       [0.14296563],
       [0.14567925],
       [0.1375907 ],
       [0.14348747]])

In [8]:
dataset_test = scaler.transform(dataset_test)
dataset_test[:7]

array([[52.27231598],
       [52.49536514],
       [53.11549377],
       [54.80430984],
       [54.68175507],
       [53.61797333],
       [53.89984894]])

In [9]:
def create_my_dataset(df):
    x = []
    y = []
    for i in range(50, df.shape[0]):
        x.append(df[i-50:i,0])
        y.append(df[i,0])
    x = np.array(x)
    y = np.array(y)
    return x,y

x_train, y_train = create_my_dataset(dataset_train)
x_test, y_test = create_my_dataset(dataset_test)

In [10]:
x_train, y_train = create_my_dataset(dataset_train)
x_train[:1]

array([[0.11896131, 0.11640431, 0.13519032, 0.14296563, 0.14567925,
        0.1375907 , 0.14348747, 0.13905185, 0.13790389, 0.14046075,
        0.13545121, 0.12991975, 0.12428404, 0.14427017, 0.14035656,
        0.14009557, 0.12647564, 0.12934574, 0.11374299, 0.11009019,
        0.09970558, 0.10414121, 0.09182603, 0.07643196, 0.08321572,
        0.08269392, 0.08989519, 0.08686852, 0.08055435, 0.09057352,
        0.08316347, 0.07241377, 0.07288345, 0.05910707, 0.04861822,
        0.02644019, 0.02910173, 0.03729443, 0.0447566 , 0.03134545,
        0.04246046, 0.02998862, 0.02753616, 0.02821453, 0.02565754,
        0.05237539, 0.04204312, 0.04491313, 0.01062865, 0.01412501]])

In [11]:
y_train[:1]

array([0.03108455])

In [12]:
x_test, y_test = create_my_dataset(dataset_test)
x_test[:1]

array([[52.27231598, 52.49536514, 53.11549377, 54.80430984, 54.68175507,
        53.61797333, 53.89984894, 54.09594345, 54.60332108, 54.1596756 ,
        53.36795807, 53.61062241, 53.35570526, 54.17682648, 53.89740372,
        53.63513184, 54.89745331, 55.0494194 , 53.66944504, 54.12535095,
        55.64258957, 55.65484619, 55.00284958, 55.64749146, 56.3975296 ,
        57.89760971, 57.81427002, 57.67946243, 57.44660568, 57.66965485,
        57.94663239, 58.95158386, 58.81677628, 59.6060257 , 59.70407104,
        60.43940735, 61.04482651, 59.63299179, 59.62563705, 60.97374344,
        62.70423126, 63.11600494, 63.02532578, 63.05228424, 63.77999115,
        63.95454407, 64.46099091, 64.40197754, 65.0190506 , 64.56916046]])

In [13]:
y_test[:1]

array([65.33621216])

In [14]:
#reshaping for LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
print(x_train.shape)
print(x_test.shape)

(957, 50, 1)
(252, 50, 1)


In [15]:
tf.logging.set_verbosity(tf.logging.ERROR)
model = Sequential()
model.add(LSTM(units=96, return_sequences=True, input_shape=(x_train.shape[1],1)))
#model.add(Dropout(0.2))
model.add(LSTM(units=96, return_sequences=True))
#model.add(Dropout(0.2))
model.add(LSTM(units=96))
#model.add(Dropout(0.2))
model.add(Dense(units=1))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 96)            37632     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 96)            74112     
_________________________________________________________________
lstm_3 (LSTM)                (None, 96)                74112     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 97        
Total params: 185,953
Trainable params: 185,953
Non-trainable params: 0
_________________________________________________________________


In [18]:
epochNum = 10
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=epochNum, batch_size=32)
model.save(os.path.join("..","static","models",f"{tickerStr}.h5"))

Epoch 1/10
957/957 [==============================] - 3s 3ms/step - loss: 0.0058
Epoch 2/10
957/957 [==============================] - 2s 2ms/step - loss: 0.0010
Epoch 3/10
957/957 [==============================] - 2s 2ms/step - loss: 8.9917e-04
Epoch 4/10
957/957 [==============================] - 2s 2ms/step - loss: 8.1232e-04
Epoch 5/10
957/957 [==============================] - 2s 2ms/step - loss: 7.9687e-04
Epoch 6/10
957/957 [==============================] - 2s 2ms/step - loss: 7.8960e-04
Epoch 7/10
957/957 [==============================] - 2s 2ms/step - loss: 7.5217e-04
Epoch 8/10
957/957 [==============================] - 2s 2ms/step - loss: 6.6689e-04
Epoch 9/10
957/957 [==============================] - 2s 2ms/step - loss: 7.4289e-04
Epoch 10/10
957/957 [==============================] - 2s 2ms/step - loss: 6.3168e-04


In [ ]:
if(not os.path.exists('C:\code\ML\mfst.h5')):
    model.fit(x_train, y_train, epochs=50, batch_size=32)
    model.save(os.join("..","static","models",f"{tickerStr}.h5"))
else:
    model = load_model('stock_prediction.h5')

In [ ]:
#visualizing our predictions
%matplotlib inline
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

fig, ax = plt.subplots(figsize=(8,4))
plt.plot(df, color='red', label='Original Stockprice')
ax.plot(range(len(y_train)+50,len(y_train)+50+len(predictions)),predictions,color='blue',label='predicted')
plt.legend()
print(range(len(y_train)+50,len(y_train)+50+len(predictions)))

In [ ]:
msft.history('1mo')['Open'].values.reshape(-1,1)

In [ ]:
predictions_prepended = [0 for i in range(0,df.shape[0]-len(predictions))]
predictions_prepended.extend(predictions.flatten().tolist())
graph1 = pd.DataFrame({
    "closing_prices": df.flatten().tolist(),
    "predicted_prices": predictions_prepended,
})
graph1.to_csv('graph_1.csv',index_label="Index")

In [ ]:
graph1.to_csv('graph_1.csv',index_label="Index")

In [ ]:
y_test_scaled = scaler.inverse_transform(y_test.reshape(-1,1))

fig, ax = plt.subplots(figsize=(8,4))
ax.plot(y_test_scaled, color='red', label='True Price of testing set')
plt.plot(predictions,color='blue',label='predicted')
plt.legend()
plt.savefig('image.png')